## Abstract Problem Formulation

**Five Components**
1. Initial States
2. Actions
3. Transition model
4. Goal test
5. Path cost

**Example:**

Component         | Abstract Example        | Real-World Detail (Ignored)
------------------|-------------------------|------------------------------
Initial State     | "At A"                  | Color of the car, weather
Actions           | "Move to B"            | Steering angle, tire pressure
Transition Model  | "A → B"                 | Traffic, road conditions
Goal Test         | "At G"                  | Driver's mood, scenery
Path Cost         | "3 steps"               | Actual fuel used, tolls


In [ ]:
class AbstractProblem:
    def __init__(self, initial_state, goal_state, graph):
        self.initial_state = initial_state
        self.goal_state = goal_state
        self.graph = graph

    def actions(self, state):
        """Returns the possible actions from the current state."""
        return self.graph.get(state, {}).keys()

    def result(self, state, action):
        """Returns the resulting state after taking an action."""
        return action

    def goal_test(self, state):
        """Checks if the state is a goal state."""
        return state == self.goal_state

    def path_cost(self, current_cost, state1, action, state2):
        """Returns the path cost from state1 to state2 via action."""
        return current_cost + self.graph[state1][action]


In [ ]:
# Example Usage

graph = {
    'S': {'A': 5, 'D': 6, 'B': 9},
    'A': {'B': 3, 'G': 9},
    'B': {'C': 1, 'A': 2},
    'C': {'D': 6, 'F': 5, 'H': 5},
    'D': {'E': 2, 'S': 1},
    'E': {'F': 7},
    'F': {'F': 8},  
    'G': {},
    'H': {},
    'I': {}  
}

 
problem = AbstractProblem(initial_state='S', goal_state='G', graph=graph)

print("Initial State:", problem.initial_state)
print("Actions from 'S':", list(problem.actions('S')))
print("Result of action 'A' from 'S':", problem.result('S', 'A'))
print("Is 'D' the goal?", problem.goal_test('D'))
print("Path cost from 'S' to 'A':", problem.path_cost(0, 'S', 'A', 'A'))


## Infrastructure of **search node**

For each node n of the tree, we have a structure that contains four components:

1. n.STATE: the state in the state space to which the node corresponds;
2. n.PARENT: the node in the search tree that generated this node;
3. n.ACTION: the action that was applied to the parent to generate the node;
3. n.PATH−COST: the cost, traditionally denoted by g(n), of the path from the initial state to the node, as indicated by the parent pointers.

Given the components for a parent node, it is easy to see how to compute the necessary components for a child node.

*function CHILD-NODE(problem, parent , action) returns a node* <br>
&nbsp;&nbsp;&nbsp;&nbsp; *return a node with*<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; *STATE = problem.RESULT(parent.STATE, action),* <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; *PARENT = parent, ACTION = action,* <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; *PATH-COST = parent.PATH-COST + problem.STEP-COST(parent.STATE, action)*

In [ ]:
class SearchNode:
    def __init__(self, state, parent=None, action=None, path_cost=0):
        self.state = state          # The state in the state space
        self.parent = parent        # The parent node in the search tree
        self.action = action        # The action taken to reach this node
        self.path_cost = path_cost  # The total cost from the initial state to this node

    
    def __lt__(self, other):
        return self.path_cost < other.path_cost

    def __repr__(self):
        return (f"SearchNode(state={self.state}, "
                f"parent={self.parent.state if self.parent else None}, "
                f"action={self.action}, path_cost={self.path_cost})")


def child_node(problem, parent, action):
    state = problem.result(parent.state, action)
    path_cost = problem.path_cost(parent.path_cost, parent.state, action, state)
    return SearchNode(state=state, parent=parent, action=action, path_cost=path_cost)

In [ ]:
# Example usage:
problem = AbstractProblem(initial_state='S', goal_state='G', graph=graph)
parent_node = SearchNode(state='S', path_cost=0)
action = 'B'
child = child_node(problem, parent_node, action)
print(child)


## Breadth First Algorithm 

function BREADTH-FIRST-SEARCH(problem) returns a solution, or failure <br>
&nbsp;&nbsp;&nbsp;&nbsp; node ← a node with STATE = problem.INITIAL-STATE, PATH-COST = 0 <br>
&nbsp;&nbsp;&nbsp;&nbsp; if problem.GOAL-TEST(node.STATE) then return SOLUTION(node) <br>
&nbsp;&nbsp;&nbsp;&nbsp; frontier ← a FIFO queue with node as the only element <br>
&nbsp;&nbsp;&nbsp;&nbsp; explored ← an empty set <br>
&nbsp;&nbsp;&nbsp;&nbsp; loop do <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; if EMPTY?(frontier) then return failure <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; node ← POP(frontier)  /* chooses the shallowest node in frontier */ <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; add node.STATE to explored <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; for each action in problem.ACTIONS(node.STATE) do <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; child ← CHILD-NODE(problem, node, action) <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; if child.STATE is not in explored or frontier then <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; if problem.GOAL-TEST(child.STATE) then return  SOLUTION(child) <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; INSERT(child, frontier) <br>


In [ ]:
from collections import deque

def breadth_first_search(problem):
    node = SearchNode(state=problem.initial_state, path_cost=0)
    if problem.goal_test(node.state):
        return node

    frontier = deque([node])
    explored = set()

    while frontier:
        node = frontier.popleft()  
        explored.add(node.state)

        for action in problem.actions(node.state):
            child = child_node(problem, node, action)
            if child.state not in explored and all(n.state != child.state for n in frontier):
                if problem.goal_test(child.state):
                    return child
                frontier.append(child)
    return None

In [ ]:
def solution(node):
    path = []
    while node:
        path.append(node.state)
        node = node.parent
    return list(reversed(path))

problem = AbstractProblem(initial_state='S', goal_state='G', graph=graph)
goal_node = breadth_first_search(problem)

if goal_node:
    print("Solution path:", solution(goal_node))
    print("Total path cost:", goal_node.path_cost)
else:
    print("Failure: No path found.")


## Depth First Search Algorithm

function DEPTH-FIRST-SEARCH(problem) returns a solution, or failure <br>
&nbsp;&nbsp;&nbsp;&nbsp; node ← a node with STATE = problem.INITIAL-STATE, PATH-COST = 0 <br>
&nbsp;&nbsp;&nbsp;&nbsp; if problem.GOAL-TEST(node.STATE) then return SOLUTION(node) <br>
&nbsp;&nbsp;&nbsp;&nbsp; frontier ← a LIFO stack with node as the only element <br>
&nbsp;&nbsp;&nbsp;&nbsp; explored ← an empty set <br>
&nbsp;&nbsp;&nbsp;&nbsp; loop do <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; if EMPTY?(frontier) then return failure <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; node ← POP(frontier)  /* chooses the deepest node in frontier */ <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; add node.STATE to explored <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; for each action in problem.ACTIONS(node.STATE) do <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; child ← CHILD-NODE(problem, node, action) <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; if child.STATE is not in explored or frontier then <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; if problem.GOAL-TEST(child.STATE) then return SOLUTION(child) <br>
                

In [ ]:
def depth_first_search(problem):
    node = SearchNode(state=problem.initial_state, path_cost=0)
    if problem.goal_test(node.state):
        return node

    frontier = [node]  # Stack for DFS
    explored = set()

    while frontier:
        node = frontier.pop()  # LIFO: pop from end
        explored.add(node.state)

        for action in problem.actions(node.state):
            child = child_node(problem, node, action)
            if child.state not in explored and all(n.state != child.state for n in frontier):
                if problem.goal_test(child.state):
                    return child
                frontier.append(child)
    return None

In [ ]:
def solution(node):
    path = []
    while node:
        path.append(node.state)
        node = node.parent
    return list(reversed(path))

problem = AbstractProblem(initial_state='S', goal_state='G', graph=graph)
goal_node = depth_first_search(problem)

if goal_node:
    print("DFS Solution path:", solution(goal_node))
    print("Total path cost:", goal_node.path_cost)
else:
    print("Failure: No path found.")


## Uniform Cost Search 

function UNIFORM-COST-SEARCH(problem) returns a solution, or failure <br>
&nbsp;&nbsp;&nbsp;&nbsp; node ← a node with STATE = problem.INITIAL-STATE, PATH-COST = 0 <br>
&nbsp;&nbsp;&nbsp;&nbsp; frontier ← a priority queue ordered by PATH-COST, with node as the only element <br>
&nbsp;&nbsp;&nbsp;&nbsp; explored ← an empty set <br>
&nbsp;&nbsp;&nbsp;&nbsp; loop do <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; if EMPTY?(frontier) then return failure <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; node ← POP(frontier)  /* chooses the node with lowest path cost */ <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; if problem.GOAL-TEST(node.STATE) then return SOLUTION(node) <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; add node.STATE to explored <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; for each action in problem.ACTIONS(node.STATE) do <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; child ← CHILD-NODE(problem, node, action) <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; if child.STATE is not in explored or frontier then <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; INSERT(child, frontier) <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; else if child.STATE is in frontier with higher PATH-COST then <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; REPLACE the existing node in frontier with child <br>

In [ ]:
import heapq

def uniform_cost_search(problem):
    node = SearchNode(state=problem.initial_state, path_cost=0)
    if problem.goal_test(node.state):
        return node

    frontier = []
    heapq.heappush(frontier, node)
    explored = set()

    while frontier:
        node = heapq.heappop(frontier)
        if problem.goal_test(node.state):
            return node
        explored.add(node.state)

        for action in problem.actions(node.state):
            child = child_node(problem, node, action)
            if child.state not in explored and all(n.state != child.state for n in frontier):
                heapq.heappush(frontier, child)
            elif any(n.state == child.state and child.path_cost < n.path_cost for n in frontier):
                # Replace node with higher cost
                frontier = [n for n in frontier if n.state != child.state]
                heapq.heappush(frontier, child)
                heapq.heapify(frontier)
    return None

In [ ]:
def solution(node):
    path = []
    while node:
        path.append(node.state)
        node = node.parent
    return list(reversed(path))

problem = AbstractProblem(initial_state='A', goal_state='D', graph=graph)
goal_node = uniform_cost_search(problem)

if goal_node:
    print("UCS Solution path:", solution(goal_node))
    print("Total path cost:", goal_node.path_cost)
else:
    print("Failure: No path found.")


## Depth Limited Search

function DEPTH-LIMITED-SEARCH(problem, limit) returns a solution, or cutoff/failure <br>
&nbsp;&nbsp;&nbsp;&nbsp; function RECURSIVE-DLS(node, problem, limit) returns a solution, or cutoff/failure <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; if problem.GOAL-TEST(node.STATE) then return SOLUTION(node) <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; else if limit == 0 then return cutoff <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; else <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; cutoff_occurred ← false <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; for each action in problem.ACTIONS(node.STATE) do <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; child ← CHILD-NODE(problem, node, action) <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; result ← RECURSIVE-DLS(child, problem, limit - 1) <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; if result == cutoff then <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; cutoff_occurred ← true <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; else if result ≠ failure then <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; return result <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; if cutoff_occurred then return cutoff <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; else return failure <br>
&nbsp;&nbsp;&nbsp;&nbsp; return RECURSIVE-DLS(MAKE-NODE(problem.INITIAL-STATE), problem, limit) <br>


In [ ]:
def recursive_dls(node, problem, limit):
    if problem.goal_test(node.state):
        return node
    elif limit == 0:
        return 'cutoff'
    else:
        cutoff_occurred = False
        for action in problem.actions(node.state):
            child = child_node(problem, node, action)
            result = recursive_dls(child, problem, limit - 1)
            if result == 'cutoff':
                cutoff_occurred = True
            elif result != 'failure':
                return result
        return 'cutoff' if cutoff_occurred else 'failure'

def depth_limited_search(problem, limit):
    root = SearchNode(state=problem.initial_state, path_cost=0)
    return recursive_dls(root, problem, limit)



In [ ]:
def solution(node):
    path = []
    while isinstance(node, SearchNode):
        path.append(node.state)
        node = node.parent
    return list(reversed(path))

problem = AbstractProblem(initial_state='A', goal_state='D', graph=graph)
limit = 3
result = depth_limited_search(problem, limit)

if isinstance(result, SearchNode):
    print("DLS Solution path:", solution(result))
    print("Total path cost:", result.path_cost)
elif result == 'cutoff':
    print("Cutoff occurred: depth limit reached.")
else:
    print("Failure: No path found.")


## Itertive Deepening Search

function ITERATIVE-DEEPENING-SEARCH(problem) returns a solution, or failure <br>
&nbsp;&nbsp;&nbsp;&nbsp; for depth from 0 to ∞ do <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; result ← DEPTH-LIMITED-SEARCH(problem, depth) <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; if result ≠ cutoff then  <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; return result


In [ ]:
def iterative_deepening_search(problem):
    depth = 0
    while True: 
        result = depth_limited_search(problem, depth)
        if isinstance(result, SearchNode):
            return result
        elif result == 'failure':
            return None
        depth += 1

In [ ]:
def solution(node):
    path = []
    while isinstance(node, SearchNode):
        path.append(node.state)
        node = node.parent
    return list(reversed(path))

problem = AbstractProblem(initial_state='A', goal_state='D', graph=graph)
goal_node = iterative_deepening_search(problem)

if goal_node:
    print("IDS Solution path:", solution(goal_node))
    print("Total path cost:", goal_node.path_cost)
else:
    print("Failure: No path found.")
